## Детекция клеток

Ваша задача: обучить YOLO для детекции дрожжевых клеток и микроструктур (см. [07_object_detection.ipynb](../workshops/07_object_detection.ipynb)). Всё необходимое для запуска обучения вы можете взять из ноутбука с практикой, доделать нужно будет самую малость:
- реализовать расчёт Mean Average Precision для всего валидационного сета
- попробовать привести regression loss к виду, который используется в Yolo9000 и YoloV3
- подобрать лучшие размеры якорных рамок с помощью кластеризации

Основная цель: любыми средствами добиться $mAP > 0.6$ на валидации.

Используйте класс `torchmetrics.detection.MeanAveragePrecision` для расчёта $mAP$.

Нас будет интересовать именно значение `map` в словаре со всеми метриками - это mean average precision, усреднённый по всем отсечкам intersection over union в диапазоне $[0.5, 0.95]$ (см. документацию к классу).

При решении можно пользоваться `lightning` или писать цикл обучения вручную. В последнем случае не забудьте вручную отправить модель и батчи на GPU, чтобы обучалось быстрее

In [21]:
from __future__ import annotations

In [1]:

! wget https://tudatalib.ulb.tu-darmstadt.de/bitstream/handle/tudatalib/3799/yeast_cell_in_microstructures_dataset.zip
! unzip yeast_cell_in_microstructures_dataset.zip -d yeast_cell_in_microstructures_dataset

--2024-10-27 03:42:36--  https://tudatalib.ulb.tu-darmstadt.de/bitstream/handle/tudatalib/3799/yeast_cell_in_microstructures_dataset.zip
Resolving tudatalib.ulb.tu-darmstadt.de (tudatalib.ulb.tu-darmstadt.de)... 130.83.152.157
Connecting to tudatalib.ulb.tu-darmstadt.de (tudatalib.ulb.tu-darmstadt.de)|130.83.152.157|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/zip]
Saving to: ‘yeast_cell_in_microstructures_dataset.zip’

yeast_cell_in_micro     [          <=>       ]  92.04M  1.12MB/s    in 72s     

Last-modified header invalid -- time-stamp ignored.
2024-10-27 03:43:49 (1.27 MB/s) - ‘yeast_cell_in_microstructures_dataset.zip’ saved [96507334]

Archive:  yeast_cell_in_microstructures_dataset.zip
   creating: yeast_cell_in_microstructures_dataset/test/
   creating: yeast_cell_in_microstructures_dataset/test/bounding_boxes/
  inflating: yeast_cell_in_microstructures_dataset/test/bounding_boxes/395.pt  
  inflating: yeast_cell_in_micr

### Задание 1 (3 балла). Цикл обучения с расчётом Mean Average Precision

Запустите обучение модели из практики на всём обучающем датасете, выведите значение $mAP$ на валидационном датасете после окончания обучения.

В этом задании добейтесь $mAP > 0.3$, если всё сделано правильно - для этого должно хватать 30-50 эпох.

In [22]:
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib.figure import Figure
from torch import Tensor, nn
from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional.detection import intersection_over_union
from torchvision.ops.boxes import box_convert


def process_yolo_preds(preds: Tensor, rescaled_anchors: Tensor) -> tuple[Tensor, Tensor, Tensor]:
    """
    Преобразование выходов модели в
    1. Логит наличия объекта (вероятность получается применением сигмоиды)
    2. Положение рамки относительно ячейки в формате cxcywh
    3. Логиты классов (вероятности получаются применением softmax)
    """
    rescaled_anchors = rescaled_anchors.view(1, len(rescaled_anchors), 1, 1, 2)
    box_predictions = preds[..., 1:5].clone()

    box_predictions[..., 0:2] = torch.sigmoid(box_predictions[..., 0:2])
    box_predictions[..., 2:] = torch.exp(box_predictions[..., 2:]) * rescaled_anchors

    scores = preds[..., 0:1]
    return scores, box_predictions, preds[..., 5:]


GRID_SIZE = 8
IMAGE_SIZE = 256
ANCHORS = [
    [48, 72],
    # [64, 64],
    # [72, 48],
]

rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
class CNNBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)

class TinyYOLO(nn.Module):
    def __init__(self, num_classes: int = 2, num_anchors: int = 1, in_channels: int = 1) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.num_anchors = num_anchors
        self.layers = nn.Sequential(
            CNNBlock(1, 16, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(16, 32, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(32, 64, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(64, 128, kernel_size=3, stride=2, padding=1, groups=8),
            CNNBlock(128, 256, kernel_size=3, stride=1, padding=1, groups=8),
            CNNBlock(256, 256, kernel_size=3, stride=1, padding=1, groups=16),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, num_anchors * (num_classes + 5), kernel_size=1)
        )
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.layers(x)
        B, _, W, H = x.shape
        x = x.view(B, self.num_anchors, self.num_classes + 5, W, H)  # B A F W H
        x = x.permute(0, 1, 3, 4, 2)  # B A W H F
        return x



In [23]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)

class TinyYOLO(nn.Module):
    def __init__(self, num_classes: int = 2, num_anchors: int = 1, in_channels: int = 1) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.num_anchors = num_anchors
        self.layers = nn.Sequential(
            CNNBlock(1, 16, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(16, 32, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(32, 64, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(64, 128, kernel_size=3, stride=2, padding=1, groups=8),
            CNNBlock(128, 256, kernel_size=3, stride=1, padding=1, groups=8),
            CNNBlock(256, 256, kernel_size=3, stride=1, padding=1, groups=16),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, num_anchors * (num_classes + 5), kernel_size=1)
        )
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.layers(x)
        B, _, W, H = x.shape
        x = x.view(B, self.num_anchors, self.num_classes + 5, W, H)  # B A F W H
        x = x.permute(0, 1, 3, 4, 2)  # B A W H F
        return x


In [24]:
def iou_wh(wh1: Tensor, wh2: Tensor) -> Tensor:
    # IoU based on width and height of bboxes

    # intersection
    intersection_area = torch.min(wh1[..., 0], wh2[..., 0]) * torch.min(wh1[..., 1], wh2[..., 1])

    # union
    box1_area = wh1[..., 0] * wh1[..., 1]
    box2_area = wh2[..., 0] * wh2[..., 1]
    union_area = box1_area + box2_area - intersection_area

    iou_score = intersection_area / union_area
    return iou_score



def boxes_to_cells(
    boxes: Tensor,
    classes: Tensor,
    rescaled_anchors: Tensor,
    grid_size: int = 8,
    ignore_iou_thresh: float = 0.5,
) -> Tensor:
    """
    Переводит bbox представление в клеточное представление, где каждая рамка -
    (id класса, вероятность нахождения объекта, cx, cy, w, h), а клеточное представление
    имеет размер (batch_size, n_anchors, grid_size, grid_size, 6), в последней размерности
    хранятся признаки ячейки: класс объекта, вероятность объекта, координаты и размеры рамки
    относительно ячейки

    Args:
        boxes (Tensor): тензор со всеми рамками
        classes (Tensor): тензор с id классов объектов
        rescaled_anchors (Tensor): тензор размера (n_anchors, 2) с размерами якорей в долях от размеров ячейки
        grid_size (int): размер сетки,
        ignore_iou_thresh (float, optional): значение IoU для рамок, при котором ячейка,
            занятая более чем одним объектом, будет специально помечена для игнорирования
    """
    targets = torch.zeros((len(rescaled_anchors), grid_size, grid_size, 6))

    # Каждой рамке сопоставляем клетку и наиболее подходящий якорь
    for box, class_label in zip(boxes, classes):
        iou_anchors = iou_wh(box[2:4], rescaled_anchors / grid_size)
        anchor_indices = iou_anchors.argsort(descending=True, dim=0)
        x, y, width, height = box

        # Относим рамку к наиболее подходящему якорю
        has_anchor = False
        for anchor_idx in anchor_indices:
            s = grid_size

            # Определяем клетку, к которой относится рамка
            i, j = int(s * y), int(s * x)
            anchor_taken = targets[anchor_idx, i, j, 0]

            # Проверяем, доступен ли якорная рамка для текущей ячейки
            if not anchor_taken and not has_anchor:
                # Пересчитываем координаты по отношению к клетке
                x_cell, y_cell = s * x - j, s * y - i
                width_cell, height_cell = (width * s, height * s)
                box_coordinates = torch.tensor([x_cell, y_cell, width_cell, height_cell])

                # Заполняем содержимое для выбранной клетки
                targets[anchor_idx, i, j, 0] = 1  # указатель, что в клетке есть объект
                targets[anchor_idx, i, j, 1:5] = box_coordinates
                targets[anchor_idx, i, j, 5] = int(class_label)

                has_anchor = True

            # Если якорь уже выбран, проверим IoU, если больше threshold - пометим клетку -1
            elif not anchor_taken and iou_anchors[anchor_idx] > ignore_iou_thresh:
                targets[anchor_idx, i, j, 0] = -1

    return targets

In [25]:
class YeastDetectionDataset(Dataset):
    def __init__(
        self, subset_dir: Path, anchors: list[tuple[int, int]], image_size: int, grid_size: int = 8
    ) -> None:
        super().__init__()
        self.subset_dir = subset_dir
        self.items = list((self.subset_dir / "inputs").glob("*.pt"))
        # Ignore IoU threshold
        self.ignore_iou_thresh = 0.5
        self.rescaled_anchors = torch.tensor(anchors) / image_size * grid_size
        self.grid_size = grid_size
        self.image_size = image_size

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, index: int) -> tuple[Tensor, Tensor]:
        image_path = self.items[index]
        # load everything
        image = torch.load(image_path, weights_only=True).unsqueeze(0)
        classes = (
            torch.load(self.subset_dir / "classes" / image_path.parts[-1], weights_only=True) + 1
        )
        boxes = torch.load(
            self.subset_dir / "bounding_boxes" / image_path.parts[-1], weights_only=True
        )
        boxes = box_convert(boxes, "xyxy", "cxcywh") / self.image_size

        # convert boxes to cells
        targets = boxes_to_cells(
            boxes, classes, self.rescaled_anchors, self.grid_size, self.ignore_iou_thresh
        )
        return image, targets
    
train_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/train"), anchors=ANCHORS, image_size=256
)
val_dataset = YeastDetectionDataset(Path("yeast_cell_in_microstructures_dataset/val"), anchors=ANCHORS, image_size=256)

batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

x, y = train_dataset[2]
x1, y1= x, y
y.shape

torch.Size([1, 8, 8, 6])

In [26]:
def cells_to_bboxes(cells: Tensor, rescaled_anchors: Tensor, is_predictions=False) -> Tensor:
    """
    Переводит клеточное представление в bbox представление, где каждая рамка -
    (id класса, вероятность нахождения объекта, cx, cy, w, h), а клеточное представление
    имеет размер (batch_size, n_anchors, grid_size, grid_size, 6), в последней размерности
    хранятся признаки ячейки: класс объекта, вероятность объекта, координаты и размеры рамки
    относительно ячейки

    Args:
        cells (Tensor): тензор размера (batch_size, n_anchors, width, height, 6)
        rescaled_anchors (Tensor): тензор размера (n_anchors, 2) с размерами якорей в долях от размеров ячейки
        is_predictions (bool, optional): являются ли входные ячейки предсказаниями или верной аннотацией.
    """

    if is_predictions:
        scores, box_predictions, logits = process_yolo_preds(cells, rescaled_anchors)
        scores = torch.sigmoid(scores)
        best_class = torch.argmax(logits, dim=-1).unsqueeze(-1) + 1

    else:
        box_predictions = cells[..., 1:5].clone()
        scores = cells[..., 0:1]
        best_class = cells[..., 5:6]

    # масштабируем размер рамок [0, grid_size] -> [0, 1]
    _, _, H, W, _ = cells.shape
    range_y, range_x = torch.meshgrid(
        torch.arange(H, dtype=cells.dtype, device=cells.device),
        torch.arange(W, dtype=cells.dtype, device=cells.device),
        indexing="ij",
    )
    x = torch.cat(
        [
            best_class,
            scores,
            (box_predictions[..., 0:1] + range_x[None, None, :, :, None]) / W,  # X center
            (box_predictions[..., 1:2] + range_y[None, None, :, :, None]) / H,  # Y center
            box_predictions[..., 2:3] / W,  # Width
            box_predictions[..., 3:4] / H,  # Height
        ],
        -1,
    )

    return x.view(-1, 6)


def plot_image(image: Tensor, boxes: Tensor, class_labels: list[str]) -> Figure:
    # назначим цвета для классов
    colour_map = plt.get_cmap("tab20b")
    colors = [colour_map(i) for i in np.linspace(0, 1, len(class_labels))]

    fig, ax = plt.subplots(1)
    ax.imshow(image, cmap="gray")
    h, w = image.shape
    for box in boxes:
        # добавляем прямоугольник
        class_pred = box[0] - 1
        box = box[2:]
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2

        rect = patches.Rectangle(
            (upper_left_x * w, upper_left_y * h),
            box[2] * w,
            box[3] * h,
            linewidth=2,
            edgecolor=colors[int(class_pred)],
            facecolor="none",
        )
        ax.add_patch(rect)

        # добавляем подпись
        ax.text(
            upper_left_x * w,
            upper_left_y * h,
            s=class_labels[int(class_pred)],
            color="white",
            verticalalignment="top",
            bbox={"color": colors[int(class_pred)], "pad": 0},
        )

    return fig


class YOLOLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, pred: Tensor, target: Tensor, anchors: Tensor) -> Tensor:
        # ниже входные тензоры будут меняться на месте, так что склонируем их
        pred = pred.clone()
        target = target.clone()

        # разделяем рамки на содержащие объекты и не содержащие
        # NB: ещё могут быть -1, куда отнеслось более 1 объекта - их не учитываем
        obj = target[..., 0] == 1
        no_obj = target[..., 0] == 0

        # преобразуем предсказания bbox
        scores, pred_boxes, logits = process_yolo_preds(pred, anchors)

        # no object loss: кросс-энтропия вместо MSE
        no_object_loss = self.bce(
            (scores[no_obj]),
            (target[..., 0:1][no_obj]),
        )

        # object loss: учим предсказывать IoU
        ious = intersection_over_union(pred_boxes[obj], target[..., 1:5][obj]).detach()
        object_loss = self.mse(scores[obj].sigmoid(), ious * target[..., 0:1][obj])

        # box coordinate loss: логарифмируем размеры рамок перед расчётом MSE
        anchors = anchors.reshape(1, len(anchors), 1, 1, 2)
        pred[..., 1:3] = pred[..., 1:3].sigmoid()
        target[..., 3:5] = torch.log(1e-6 + target[..., 3:5] / anchors)
        box_loss = self.mse(pred[..., 1:5][obj], target[..., 1:5][obj])

        # class loss: здесь всё обычно
        class_loss = self.cross_entropy(logits[obj], target[..., 5][obj].long() - 1)

        # Total loss
        return box_loss + object_loss + no_object_loss + class_loss
rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
torch.manual_seed(42)

model = TinyYOLO(in_channels=1, num_classes=2, num_anchors=len(rescaled_anchors))
optim = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
loss_fn = YOLOLoss()
for epoch in range(40):
    model.train()
    for x, y in iter(train_loader):
        pred = model(x)
        loss= loss_fn(pred, y, rescaled_anchors)
        loss.backward()
        optim.step()
        optim.zero_grad()
        
    print(f"Epoch {epoch+1}, loss: {loss:.4f}")
        
    
    

Epoch 1, loss: 0.3803
Epoch 2, loss: 0.2765
Epoch 3, loss: 0.1627
Epoch 4, loss: 0.1647
Epoch 5, loss: 0.1063
Epoch 6, loss: 0.1429
Epoch 7, loss: 0.0794
Epoch 8, loss: 0.0586
Epoch 9, loss: 0.0871
Epoch 10, loss: 0.0658
Epoch 11, loss: 0.0768
Epoch 12, loss: 0.0780
Epoch 13, loss: 0.0604
Epoch 14, loss: 0.0481
Epoch 15, loss: 0.0849
Epoch 16, loss: 0.0395
Epoch 17, loss: 0.0337
Epoch 18, loss: 0.0420
Epoch 19, loss: 0.0442
Epoch 20, loss: 0.0308
Epoch 21, loss: 0.0570
Epoch 22, loss: 0.0257
Epoch 23, loss: 0.0365
Epoch 24, loss: 0.0217
Epoch 25, loss: 0.0354
Epoch 26, loss: 0.0246
Epoch 27, loss: 0.0269
Epoch 28, loss: 0.0317
Epoch 29, loss: 0.0334
Epoch 30, loss: 0.0251
Epoch 31, loss: 0.0203
Epoch 32, loss: 0.0469
Epoch 33, loss: 0.0185
Epoch 34, loss: 0.0223
Epoch 35, loss: 0.0160
Epoch 36, loss: 0.0176
Epoch 37, loss: 0.0364
Epoch 38, loss: 0.0237
Epoch 39, loss: 0.0174
Epoch 40, loss: 0.0175


In [27]:
from torchmetrics.detection import MeanAveragePrecision
import torchmetrics
import pycocotools

torch.manual_seed(42)
def process_boxes(boxes, image_size=256):
    """
    Преобразует рамки из формата cx, cy, w, h в формат xyxy в пикселях.
    """
    if boxes.numel() == 0:
        return torch.empty((0, 4), dtype=boxes.dtype, device=boxes.device)
    cxcy = boxes[:, 0:2]
    wh = boxes[:, 2:4]
    x1y1 = cxcy - wh / 2
    x2y2 = cxcy + wh / 2
    xyxy = torch.cat([x1y1, x2y2], dim=1) * image_size
    return xyxy

model.eval()
metric = MeanAveragePrecision()

with torch.no_grad():
    for batch_idx, (x_val, y_val) in enumerate(val_loader):
        pred = model(x_val)
        batch_size = x_val.shape[0]
        for i in range(batch_size):
            pred_cells = pred[i]
            target_cells = y_val[i]
            preds = cells_to_bboxes(pred_cells.unsqueeze(0), rescaled_anchors, is_predictions=True)
            scores = preds[:, 1]
            conf_threshold = 0.5
            preds = preds[scores > conf_threshold]
            targets = cells_to_bboxes(target_cells.unsqueeze(0), rescaled_anchors, is_predictions=False)
            targets = targets[targets[:, 1] == 1]
            
            if preds.shape[0] == 0 or targets.shape[0] == 0:
                continue
            
            #print(f"Batch {batch_idx}, Image {i}: preds shape: {preds.shape}, targets shape: {targets.shape}")
            
            if preds.shape[1] != 6 or targets.shape[1] != 6:
                print(f"Неверная форма данных для Batch {batch_idx}, Image {i}")
                continue
            
            pred_boxes = {
                'boxes': process_boxes(preds[:, 2:6]),
                'scores': preds[:, 1],
                'labels': (preds[:, 0].long() - 1)
            }
            target_boxes = {
                'boxes': process_boxes(targets[:, 2:6]),
                'labels': (targets[:, 0].long() - 1)
            }
            
            if pred_boxes['boxes'].shape[0] == 0 or target_boxes['boxes'].shape[0] == 0:
                print(f"Пустые рамки после обработки для Batch {batch_idx}, Image {i}")
                continue
            
            metric.update([pred_boxes], [target_boxes])

result = metric.compute()
print(f"mAP на валидационном сете: {result['map']:.4f}")

mAP на валидационном сете: 0.3839


### Задание 2 (2 балла). YoloV3 loss

Мы упоминали, что на практике использовалась не совсем та же самая ошибка, что и в YOLO. В этом задании исправьте в классе YoloLoss ошибку регрессии, приведя её в соответствие с тем, как она описана в статье [YOLOv3: An Incremental Improvement](https://arxiv.org/abs/1804.02767) (см. раздел 2.1. Bounding Box Prediction).

Запустите обучение с изменённой ошибкой, добейтесь $mAP > 0.3$

In [30]:
class YOLOLoss2(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, pred: Tensor, target: Tensor, anchors: Tensor) -> Tensor:
        # ниже входные тензоры будут меняться на месте, так что склонируем их
        pred = pred.clone()
        target = target.clone()

        # разделяем рамки на содержащие объекты и не содержащие
        # NB: ещё могут быть -1, куда отнеслось более 1 объекта - их не учитываем
        obj = target[..., 0] == 1
        no_obj = target[..., 0] == 0

        # преобразуем предсказания bbox
        scores, pred_boxes, logits = process_yolo_preds(pred, anchors)

        # no object loss: кросс-энтропия вместо MSE
        no_object_loss = self.bce(
            (scores[no_obj]),
            (target[..., 0:1][no_obj]),
        )

        # object loss: учим предсказывать IoU
        ious = intersection_over_union(pred_boxes[obj], target[..., 1:5][obj]).detach()
        object_loss = self.mse(scores[obj].sigmoid(), ious * target[..., 0:1][obj])

        # box coordinate loss: логарифмируем размеры рамок перед расчётом MSE
        anchors = anchors.reshape(1, len(anchors), 1, 1, 2)
        pred[..., 1:3] = pred[..., 1:3].sigmoid()
        target[..., 3:5] = torch.log(1e-6 + target[..., 3:5] / anchors)
        #box_loss = self.mse(pred[..., 1:5][obj], target[..., 1:5][obj])
        box_loss = nn.MSELoss(reduction='sum')(pred[..., 1:5][obj], target[..., 1:5][obj])* 0.3

        # class loss: здесь всё обычно
        class_loss = self.cross_entropy(logits[obj], target[..., 5][obj].long() - 1)

        # Total loss
        return box_loss + object_loss + no_object_loss + class_loss
    
    
    
#rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
torch.manual_seed(42)

model1 = TinyYOLO(in_channels=1, num_classes=2, num_anchors=len(rescaled_anchors))
optim1 = torch.optim.Adam(model1.parameters(), lr=0.001, weight_decay=0.001)
loss_fn1 = YOLOLoss2()
for epoch in range(40):
    model1.train()
    for x1, y1 in iter(train_loader):
        pred1 = model1(x1)
        loss = loss_fn1(pred1, y1, rescaled_anchors)
        loss.backward()
        optim1.step()
        optim1.zero_grad()
        
    print(f"Epoch {epoch+1}, loss: {loss:.4f}")
    
    
from torchmetrics.detection import MeanAveragePrecision
import torchmetrics
import pycocotools

torch.manual_seed(42)
def process_boxes(boxes, image_size=256):
    """
    Преобразует рамки из формата cx, cy, w, h в формат xyxy в пикселях.
    """
    if boxes.numel() == 0:
        return torch.empty((0, 4), dtype=boxes.dtype, device=boxes.device)
    cxcy = boxes[:, 0:2]
    wh = boxes[:, 2:4]
    x1y1 = cxcy - wh / 2
    x2y2 = cxcy + wh / 2
    xyxy = torch.cat([x1y1, x2y2], dim=1) * image_size
    return xyxy

model1.eval()
metric1 = MeanAveragePrecision()

with torch.no_grad():
    for batch_idx, (x_val, y_val) in enumerate(val_loader):
        pred1 = model1(x_val)
        batch_size = x_val.shape[0]
        for i in range(batch_size):
            pred_cells = pred1[i]
            target_cells = y_val[i]
            preds = cells_to_bboxes(pred_cells.unsqueeze(0), rescaled_anchors, is_predictions=True)
            scores = preds[:, 1]
            conf_threshold = 0.5
            preds = preds[scores > conf_threshold]
            targets = cells_to_bboxes(target_cells.unsqueeze(0), rescaled_anchors, is_predictions=False)
            targets = targets[targets[:, 1] == 1]
            
            if preds.shape[0] == 0 or targets.shape[0] == 0:
                continue
            
            #print(f"Batch {batch_idx}, Image {i}: preds shape: {preds.shape}, targets shape: {targets.shape}")
            
            if preds.shape[1] != 6 or targets.shape[1] != 6:
                print(f"Неверная форма данных для Batch {batch_idx}, Image {i}")
                continue
            
            pred_boxes = {
                'boxes': process_boxes(preds[:, 2:6]),
                'scores': preds[:, 1],
                'labels': (preds[:, 0].long() - 1)
            }
            target_boxes = {
                'boxes': process_boxes(targets[:, 2:6]),
                'labels': (targets[:, 0].long() - 1)
            }
            
            if pred_boxes['boxes'].shape[0] == 0 or target_boxes['boxes'].shape[0] == 0:
                print(f"Пустые рамки после обработки для Batch {batch_idx}, Image {i}")
                continue
            
            metric1.update([pred_boxes], [target_boxes])

result1 = metric1.compute()
print(f"mAP на валидационном сете: {result1['map']:.4f}")

Epoch 1, loss: 7.9720
Epoch 2, loss: 4.1946
Epoch 3, loss: 2.0155
Epoch 4, loss: 1.6354
Epoch 5, loss: 1.2293
Epoch 6, loss: 1.4041
Epoch 7, loss: 0.8645
Epoch 8, loss: 0.9074
Epoch 9, loss: 1.0053
Epoch 10, loss: 0.9327
Epoch 11, loss: 0.9161
Epoch 12, loss: 0.7178
Epoch 13, loss: 0.6209
Epoch 14, loss: 0.4568
Epoch 15, loss: 0.6584
Epoch 16, loss: 0.4297
Epoch 17, loss: 0.4271
Epoch 18, loss: 0.5608
Epoch 19, loss: 0.5785
Epoch 20, loss: 0.2923
Epoch 21, loss: 0.3434
Epoch 22, loss: 0.4190
Epoch 23, loss: 0.4095
Epoch 24, loss: 0.2776
Epoch 25, loss: 0.3954
Epoch 26, loss: 0.2800
Epoch 27, loss: 0.5385
Epoch 28, loss: 0.2846
Epoch 29, loss: 0.2828
Epoch 30, loss: 0.3334
Epoch 31, loss: 0.3405
Epoch 32, loss: 0.4422
Epoch 33, loss: 0.2896
Epoch 34, loss: 0.3105
Epoch 35, loss: 0.2240
Epoch 36, loss: 0.2183
Epoch 37, loss: 0.2954
Epoch 38, loss: 0.2766
Epoch 39, loss: 0.2244
Epoch 40, loss: 0.2385
mAP на валидационном сете: 0.3242


### Задание 3 (3 балла). Выбор anchors с помощью кластеризации

В статье [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) в разделе 2. Better. Dimension clusters описан способ выбора anchor boxes через кластеризацию обучающего датасета.

Проделайте то же самое с вашим обучающим датасетом, чтобы выбрать несколько anchor boxes.

В качестве результата выведите получившиеся размеры anchors для # Clusters = 5

### Задание 4 (4 балла + бонусы за лучшую точность). Обучите модель


Ваша цель: $mAP > 0.6$ на валидации.

Можете использовать весь арсенал:
- использование множества якорных рамок (сформированных вручную или в результате кластеризации)
- любые изменения функции ошибки
- любые изменения архитектуры модели и регуляризация
- аугментации (вспоминаем `torchvision.transforms` и `albumentations`)
- любая длительность обучения, оптимизатор, расписание для learning rate

Бонусы за повышенную точность:
- **5 баллов**: $mAP > 0.65$
- **1 балл** за каждые следующие $0.01$ (т. е. за $mAP > 0.72$ в этом задании вы получите $4 + 12 = 16$ баллов)

**Важно**: перез запуском обучения зафиксируйте `torch.manual_seed()`

In [39]:
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.GaussNoise(p=0.2),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='coco', label_fields=['labels']))
ANCHORS = [
    [ 71.96787,  105.064255],
    [ 76.63636,  68.74545 ],
    [ 55.532753,  62.39738 ],
    [ 57.544117,  87.911766],
    [ 43.41379 ,  45.23448 ]
]



class YeastDetectionDataset(Dataset):
    def __init__(
        self,
        subset_dir: Path,
        anchors: list[tuple[int, int]],
        image_size: int,
        grid_size: int = 8,
        transform=None 
    ) -> None:
        super().__init__()
        self.subset_dir = subset_dir
        self.items = list((self.subset_dir / "inputs").glob("*.pt"))
        # Ignore IoU threshold
        self.ignore_iou_thresh = 0.5
        self.rescaled_anchors = torch.tensor(anchors) / image_size * grid_size
        self.grid_size = grid_size
        self.image_size = image_size
        self.transform = transform 

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, index: int) -> tuple[Tensor, Tensor]:
        image_path = self.items[index]
    # load everything
        image = torch.load(image_path, weights_only=True).unsqueeze(0)
        classes = (
            torch.load(self.subset_dir / "classes" / image_path.parts[-1], weights_only=True) + 1
        )   
        boxes = torch.load(
             self.subset_dir / "bounding_boxes" / image_path.parts[-1], weights_only=True
        )
        boxes = box_convert(boxes, "xyxy", "cxcywh") / self.image_size

        # Преобразование боксов в формат для albumentations
        boxes_xywh = boxes * self.image_size  # Преобразуем координаты в пиксели
        boxes_xywh = box_convert(boxes_xywh, 'cxcywh', 'xywh')
        labels = classes.numpy().tolist()

    # Применение аугментаций, если они заданы
        if self.transform:
            transformed = self.transform(
                image=image.squeeze(0).numpy(),
                bboxes=boxes_xywh.tolist(),
             labels=labels
          )
            image = transformed['image']
            boxes = torch.tensor(transformed['bboxes'])
            boxes = box_convert(boxes, 'xywh', 'cxcywh') / self.image_size  # Нормализуем обратно
            classes = torch.tensor(transformed['labels'])

        else:
             image = torch.tensor(image, dtype=torch.float32)
             boxes = boxes
        classes = classes
        
    # convert boxes to cells
        targets = boxes_to_cells(
              boxes, classes, self.rescaled_anchors, self.grid_size, self.ignore_iou_thresh
        )
        return image, targets
    

rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
class TinyYOLO(nn.Module):
    def __init__(self, num_classes: int = 2, num_anchors: int = 1, in_channels: int = 1) -> None:
        super(TinyYOLO, self).__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors
        self.in_channels = in_channels

        self.layer1 = CNNBlock(in_channels, 16, kernel_size=3, stride=2, padding=1, dilation=2)
        self.layer2 = CNNBlock(16, 32, kernel_size=3, stride=2, padding=1, dilation=2)
        self.layer3 = CNNBlock(32, 64, kernel_size=3, stride=2, padding=1, dilation=2)
        self.layer4 = CNNBlock(64, 128, kernel_size=3, stride=2, padding=1, groups=8)
        self.layer5 = CNNBlock(128, 256, kernel_size=3, stride=1, padding=1, groups=8)
        self.layer6 = CNNBlock(256, 256, kernel_size=3, stride=1, padding=1, groups=8)
        self.layer7 = CNNBlock(256, 512, kernel_size=3, stride=1, padding=1, groups=4)
        self.layer8 = CNNBlock(512, 512, kernel_size=3, stride=1, padding=1, groups=8)

        # 1x1 convolutions for skip connections
        self.skip_conv1 = nn.Conv2d(16, 64, kernel_size=1)
        self.skip_conv2 = nn.Conv2d(32, 128, kernel_size=1)
        self.skip_conv3 = nn.Conv2d(64, 512, kernel_size=1)
        self.skip_conv4 = nn.Conv2d(128, 256, kernel_size=1)

        self.downsample = nn.MaxPool2d(2, 2)
        self.output_conv = nn.Conv2d(512, num_anchors * (num_classes + 5), kernel_size=1)

    def forward(self, x: Tensor) -> Tensor:
        feat1 = self.layer1(x)
        feat2 = self.layer2(feat1)
        feat3 = self.layer3(feat2)

        # Skip connection from feat1
        resized_feat1 = F.interpolate(feat1, size=feat3.shape[2:])
        resized_feat1 = self.skip_conv1(resized_feat1)
        feat4 = self.layer4(feat3 + resized_feat1)

        # Skip connection from feat2
        resized_feat2 = F.interpolate(feat2, size=feat4.shape[2:])
        resized_feat2 = self.skip_conv2(resized_feat2)
        feat5 = self.layer5(feat4 + resized_feat2)

        # Skip connection from feat4
        resized_feat4 = F.interpolate(feat4, size=feat5.shape[2:])
        resized_feat4 = self.skip_conv4(resized_feat4)
        feat6 = self.layer6(feat5 + resized_feat4)

        feat7 = self.layer7(feat6)

        # Skip connection from feat3
        resized_feat3 = F.interpolate(feat3, size=feat7.shape[2:])
        resized_feat3 = self.skip_conv3(resized_feat3)
        feat8 = self.layer8(feat7 + resized_feat3)

        output = self.output_conv(self.downsample(feat8))

        B, _, W, H = output.shape
        output = output.view(B, self.num_anchors, self.num_classes + 5, W, H)
        output = output.permute(0, 1, 3, 4, 2)

        return output
    
class CNNBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LogSigmoid()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)
    
    
    
train_dataset = YeastDetectionDataset(
    subset_dir=Path("yeast_cell_in_microstructures_dataset/train"),
    anchors=ANCHORS,
    image_size=IMAGE_SIZE,
    transform=train_transforms  
)
val_transforms = A.Compose([
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='coco', label_fields=['labels']))

val_dataset = YeastDetectionDataset(
    subset_dir=Path("yeast_cell_in_microstructures_dataset/val"),
    anchors=ANCHORS,
    image_size=IMAGE_SIZE,
    transform=val_transforms  # Передаём трансформации для валидации
)
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    
model = TinyYOLO(in_channels=1, num_classes=2, num_anchors=len(rescaled_anchors))
optim = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.1, patience=5, verbose=True)
loss_fn = YOLOLoss()

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for x, y in train_loader:
        pred = model(x)
        loss = loss_fn(pred, y, rescaled_anchors)
        loss.backward()
        optim.step()
        optim.zero_grad()
        epoch_loss += loss.item()
    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}, loss: {avg_loss:.4f}")

    # Обновление планировщика
    scheduler.step(avg_loss)
    
    
    from torchvision.ops.boxes import nms

with torch.no_grad():
    for batch_idx, (x_val, y_val) in enumerate(val_loader):
        pred = model(x_val)
        batch_size = x_val.shape[0]
        for i in range(batch_size):
            pred_cells = pred[i]
            target_cells = y_val[i]
            preds = cells_to_bboxes(pred_cells.unsqueeze(0), rescaled_anchors, is_predictions=True)
            scores = preds[:, 1]
            conf_threshold = 0.5
            preds = preds[scores > conf_threshold]

            # Применяем NMS
            if preds.shape[0] > 0:
                keep_indices = nms(
                    boxes=process_boxes(preds[:, 2:6]),
                    scores=preds[:, 1],
                    iou_threshold=0.3
                )
                preds = preds[keep_indices]

            targets = cells_to_bboxes(target_cells.unsqueeze(0), rescaled_anchors, is_predictions=False)
            targets = targets[targets[:, 1] == 1]

            if preds.shape[0] == 0 or targets.shape[0] == 0:
                continue

            pred_boxes = {
                'boxes': process_boxes(preds[:, 2:6]),
                'scores': preds[:, 1],
                'labels': (preds[:, 0].long() - 1)
            }
            target_boxes = {
                'boxes': process_boxes(targets[:, 2:6]),
                'labels': (targets[:, 0].long() - 1)
            }

            metric.update([pred_boxes], [target_boxes])

    result = metric.compute()
    print(f"mAP на валидационном сете: {result['map']:.4f}")


TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor)!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, Tensor, Tensor, Tensor, Ten